In [1]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

# Struct2lar(struct) 

it turn the stuct into is the linear algebraic representation as a tuple(vertices,faces(cells),edges). it basicaly flat the struct into a unique data struct of lar type.

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
table = Lar.apply(Lar.t(-0.5,-0.5),square)

([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])

In [4]:
structure = Lar.Struct([repeat([table,Lar.r(pi/2)],outer=2)...])

LinearAlgebraicRepresentation.Struct(Any[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0], ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0]], [[-0.5; -0.5], [0.5; 0.5]], "17680028407428455000", 2, "feature")

In [9]:
function struct2lar(structure)
	listOfModels = Lar.evalStruct(structure)
	vertDict = Dict()
	index,defaultValue = 0,0
	W = Array{Float64,1}[]
	m = length(listOfModels[1])
	larmodel = [Array{Number,1}[] for k=1:m]

	for model in listOfModels
		V = model[1]
		for k=2:m
			for incell in model[k]
				outcell=[]
				for v in incell
					key = map(Lar.approxVal(7), V[:,v])
					if get(vertDict,key,defaultValue)==defaultValue
						index += 1
						vertDict[key]=index
						push!(outcell,index)
						push!(W,key)
					else
						push!(outcell,vertDict[key])
					end
				end
				append!(larmodel[k],[outcell])
			end
		end
	end

	append!(larmodel[1], W)
	V = hcat(larmodel[1]...)
	chains = [convert(Lar.Cells, chain) for chain in larmodel[2:end]]
	return (V, chains...)
end

struct2lar (generic function with 1 method)

In [6]:
@btime struct2lar(structure)

  27.500 μs (139 allocations: 8.53 KiB)


([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4], [3, 1, 4, 2]])

In [10]:
@benchmark struct2lar(structure)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  23.700 μs …  5.994 ms  ┊ GC (min … max): 0.00% … 98.39%
 Time  (median):     25.200 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   29.594 μs ± 84.054 μs  ┊ GC (mean ± σ):  3.93% ±  1.39%

  ▅█▆▄▄▃▂▁▁▁▁▁▁▂▃▂▁                                           ▁
  ███████████████████▇▆▆▆▆▅▆▆▄▃▆▅▅▅▄▅▅▆▆█▇▆▆▅▅▆▆▆▅▅▅▄▅▄▄▅▅▂▄▆ █
  23.7 μs      Histogram: log(frequency) by time      70.3 μs <

 Memory estimate: 8.53 KiB, allocs estimate: 139.

# Improvment and changes

In [13]:
function struct2lar(structure) 
	larmodel,W = flat(Lar.evalStruct(structure))
	append!(larmodel[1], W)
	V = hcat(larmodel[1]...)
	chains = [convert(Lar.Cells, chain) for chain in larmodel[2:end]]
	return (V, chains...)
end

struct2lar (generic function with 1 method)

In [14]:
function flat(listOfModels)
	W = Array{Float64,1}[]
	m = length(listOfModels[1])
	larmodel = [Array{Float64,1}[] for k=1:m]
	vertDict = Dict()
	index,defaultValue = 0,0
	for model in listOfModels
		V = model[1]
		for k=2:m
			for incell in model[k]
				outcell=[]
				for v in incell
					key = map(Lar.approxVal(7), V[:,v])
					if get(vertDict,key,defaultValue)==defaultValue
						index += 1
						vertDict[key]=index
						push!(outcell,index)
						push!(W,key)
					else
						push!(outcell,vertDict[key])
					end
				end
				append!(larmodel[k],[outcell])
			end
		end
	end
	return larmodel,W
end

flat (generic function with 1 method)

In [15]:
@benchmark struct2lar(structure)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  22.200 μs …  6.255 ms  ┊ GC (min … max): 0.00% … 98.50%
 Time  (median):     23.400 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   26.081 μs ± 62.601 μs  ┊ GC (mean ± σ):  2.36% ±  0.99%

  ▅█▇▄▄▄▄▂▂▁▁▂▁ ▁▂▃▂▁                                         ▁
  █████████████████████▇▆▆▆▆▅▆▅▅▅▆▆▅▅▄▄▄▅▄▄▅▃▅▃▅▅▄▄▃▅▂▅▅▄▄▅▄▅ █
  22.2 μs      Histogram: log(frequency) by time      56.8 μs <

 Memory estimate: 7.73 KiB, allocs estimate: 120.